In [1]:
from agent import TelosAgent
from environment import TelosTaskEnv, make_env
from task import TelosTask
from time import sleep
import pybullet as p

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3 import SAC, PPO


pybullet build time: Nov 28 2023 23:45:17


In [2]:
from standing_task.standing_task import StandingTelosTask
from standing_task.standing_environment import StandingTelosTaskEnv

In [3]:
import RMP
import utils.stability_functions as sf
import utils.target_functions as tfun
from utils.PyBullet import PyBullet
from utils.helper import load_yaml

In [4]:
pb = PyBullet(render_mode="rgb_array",renderer="OpenGL")
telos_agent = TelosAgent(pb)
telos_task = StandingTelosTask(agent=telos_agent,sim_engine=pb)
telos_env = StandingTelosTaskEnv(task=telos_task, agent=telos_agent, sim_engine=pb)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (Intel(R) Iris(R) Xe Graphics)
GL_VERSION=4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (Intel(R) Iris(R) Xe Graphics)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation


numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
finished


In [5]:
telos_task.sim.get_pitch_angle(telos_task.agent.robot_agent),telos_task.sim.get_roll_angle(telos_task.agent.robot_agent),telos_task.sim.get_yaw_angle(telos_task.agent.robot_agent)

(-0.0, 1.5707963267948963, -3.141592653589793)

In [6]:
test_model = SAC.load("sac_standing_task")

In [7]:
obs,_ = telos_env.reset()

In [8]:
obs["agent"]

array([ 0.0000000e+00,  0.0000000e+00,  3.0000001e-01, -4.3297803e-17,
        7.0710677e-01,  7.0710677e-01, -4.3297803e-17,  0.0000000e+00,
        0.0000000e+00, -7.8539819e-01,  0.0000000e+00, -1.4835298e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -7.8539819e-01,
        0.0000000e+00, -1.4835298e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00, -7.8539819e-01,  0.0000000e+00, -1.4835298e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -7.8539819e-01,
        0.0000000e+00, -1.4835298e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00], dtype=float32)

In [7]:
for i in range(2):
    obs,_ = telos_env.reset()
    done = False
    while not done:
        action,_ = test_model.predict(obs, deterministic=True)
        obs, rewards, done,_, info = telos_env.step(action)
        if done:
            sleep(8)
        telos_env.render()
        sleep(1/240)
    print(f"Episode {i} done")

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished


KeyboardInterrupt: 

In [5]:
env = DummyVecEnv([lambda: telos_env])
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [7]:
import time
from tqdm import tqdm

In [54]:
telos_env.reset()
angular_velocities = []

In [55]:
for _ in tqdm(range(1_000)):
    action = telos_env.action_space.sample()
    obs, _, _, _, _ = telos_env.step(action)
    obs = obs["agent"]
    # first 7 values are the joint positions, next 12 values are angular velocities, the shape is 34
    # print(obs,"\n",obs[7:19])
    angular_velocities.extend(obs[7:19].flatten().tolist())
    # for x in obs[7:19]:
    #     angular_velocities.append(x)
    # time.sleep(1/240)
    

100%|██████████| 1000/1000 [00:01<00:00, 995.00it/s]


In [56]:
import numpy as np

In [57]:
angular_velocities

[0.1837201565504074,
 1.7593454122543335,
 -0.9064911007881165,
 -1.146234154701233,
 -0.6068150401115417,
 5.234973907470703,
 -0.1735190451145172,
 -1.9430850744247437,
 -0.8330462574958801,
 -0.8723039627075195,
 -0.07908421754837036,
 11.097297668457031,
 -0.1497141271829605,
 -1.9860985279083252,
 0.14524954557418823,
 7.25466775894165,
 -0.770643413066864,
 -1.0253808498382568,
 -0.31734466552734375,
 -0.8604647517204285,
 -1.0005297660827637,
 -0.7083649039268494,
 -0.37109318375587463,
 -1.755929946899414,
 -0.09036701172590256,
 0.21772052347660065,
 -1.5627622604370117,
 -10.564516067504883,
 -0.7280144691467285,
 0.24617625772953033,
 0.2473500370979309,
 3.53613018989563,
 -0.5566127896308899,
 3.360050916671753,
 -1.3329273462295532,
 -5.777561187744141,
 -0.606072187423706,
 -3.028050422668457,
 -0.15973132848739624,
 10.241861343383789,
 -1.2626347541809082,
 -3.093356132507324,
 -0.44049447774887085,
 -4.265799522399902,
 -1.5620150566101074,
 -7.557089328765869,
 -0.13

In [59]:
max(angular_velocities), min(angular_velocities), np.mean(angular_velocities)

(23.861408233642578, -14.774767875671387, -0.31727150375772845)

In [12]:
act = env.action_space.sample()
env.step(act)

KeyboardInterrupt: 

In [7]:
for _ in range(1000):
    env.step(env.action_space.sample())

KeyboardInterrupt: 

In [7]:
# Saving the statistics
env.save("vec_normalize_stats.pkl")

# Loading the statistics
env = DummyVecEnv([lambda: telos_env])
env = VecNormalize.load("vec_normalize_stats.pkl", env)

In [8]:
obs = env.reset()
print(f"Initial normalized observation: {obs}")


Initial normalized observation: OrderedDict([('agent', array([[ 0.0000000e+00,  0.0000000e+00,  2.8732154e-03, -4.3293473e-19,
         5.7731178e-03,  5.7731178e-03, -4.3293473e-19,  0.0000000e+00,
         0.0000000e+00, -6.1762962e-03,  0.0000000e+00, -8.2918098e-03,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -6.1762962e-03,
         0.0000000e+00, -8.2918098e-03,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00, -6.1762962e-03,  0.0000000e+00, -8.2918098e-03,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -6.1762962e-03,
         0.0000000e+00, -8.2918098e-03,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00]], dtype=float32))])


In [13]:
import numpy as np

obs_mean = []
for _ in range(10000):
    obs = env.reset()['agent']
    obs_mean.append(obs[0])

obs_mean = np.mean(obs_mean, axis=0)
obs_var = np.var(obs_mean, axis=0)

print(f"Mean of observations: {obs_mean}")
print(f"Variance of observations: {obs_var}")


Mean of observations: [ 0.0000000e+00  0.0000000e+00  2.9958108e-05 -4.4468838e-21
  6.3219712e-05  6.3219712e-05 -4.4468838e-21  0.0000000e+00
  0.0000000e+00 -6.8321628e-05  0.0000000e+00 -9.8436787e-05
  0.0000000e+00  0.0000000e+00  0.0000000e+00 -6.8321628e-05
  0.0000000e+00 -9.8436787e-05  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -6.8321628e-05  0.0000000e+00 -9.8436787e-05
  0.0000000e+00  0.0000000e+00  0.0000000e+00 -6.8321628e-05
  0.0000000e+00 -9.8436787e-05  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
Variance of observations: 1.7250718631345308e-09


In [6]:
telos_env.sim.step_simulation()

In [7]:
telos_env.step(telos_env.action_space.sample())

({'agent': array([ 0.01513068, -0.02063485,  0.35997656,  0.02317744,  0.8097507 ,
          0.5516136 ,  0.19871849, -1.5860538 , -9.792909  , -0.499087  ,
          1.9992552 , -0.7209957 ,  5.0808544 ,  0.28545693,  1.6371651 ,
         -1.6162144 , -5.298197  , -0.50405395,  6.589702  , -1.0643768 ,
         -6.5271    , -1.1462687 , -2.1386375 , -1.1456615 ,  2.5393438 ,
         -1.4613614 , -9.241836  , -1.6811948 , -5.6025567 , -1.4512823 ,
          1.1472082 ,  0.07196213, -0.21193391,  0.56491333], dtype=float32)},
 -95.2466382172834,
 True,
 False,
 {'agent_position': array([ 0.01513068, -0.02063485,  0.35997656], dtype=float32),
  'goal_position': array([0.  , 0.  , 0.26], dtype=float32)})

In [8]:
_config = load_yaml("pybullet_config.yaml")

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished


In [ ]:
_config["task"]["fall_threshold"]

In [ ]:

Kp = _config["stability_policy"]["Kp"]
Kd = _config["stability_policy"]["Kd"]

z_position_emphasis = _config["stability_metric"]["z_position_emphasis"]
pitch_emphasis = _config["stability_metric"]["pitch_emphasis"]

In [ ]:
stability_rmp = RMP.RMP(
    sf.get_stability_task_map_from_obs,
    sf.stability_policy,
    sf.stability_metric,
    Kp=Kp,
    Kd=Kd,
    z_position_emphasis=z_position_emphasis,
    pitch_emphasis=pitch_emphasis)
target_rmp = RMP.RMP(
    lambda obs: sf.target_task_map(obs, telos_env.task.goal),
    tfun.target_policy,
    tfun.target_metric,
    Kp=Kp,
    Kd=Kd,
    z_position_emphasis=z_position_emphasis,
    pitch_emphasis=pitch_emphasis
)
global_rmp_policy = RMP.GlobalRMPPolicy(
    [stability_rmp, target_rmp]
)

In [ ]:
env = RMP.RMPRewardWrapper(telos_env, global_rmp_policy)

In [ ]:
model = PPO('MultiInputPolicy', telos_env, verbose=1)
model.learn(total_timesteps=50)

In [ ]:
env.step(telos_env.action_space.sample())

---

In [ ]:
telos_agent = TelosAgent(render_mode="rgb_array",renderer="Tiny")
telos_task = TelosTask(agent=telos_agent)
telos_env = TelosTaskEnv(task=telos_task, agent=telos_agent)

In [ ]:
env = make_vec_env(make_env, n_envs=4, env_kwargs={"task": telos_task, "agent": telos_agent, "render_mode": "rgb_array"})
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [ ]:
policy_sac =  dict(net_arch=dict(pi=[512, 512,512], qf=[512, 512,512]))

In [ ]:
model_sac = SAC(
    policy="MultiInputPolicy",
    env=env,
    policy_kwargs=policy_sac,
    learning_rate= 3.5e-4,
    buffer_size= int(1e6),
    batch_size= 2048,
    ent_coef= "auto",
    gamma= 0.99,
    tau= 0.005,
    train_freq= 1,
    gradient_steps= 1,
    action_noise= None,
    optimize_memory_usage= False,
    target_update_interval= 1,
    verbose= 1,
    tensorboard_log="./tensorboard/sacTelos/",
)

In [ ]:
model_sac.learn(500_000)

In [ ]:
for _ in range(1000):
    # print(telos_task.compute_reward(telos_agent._get_obs()[0:3], telos_task.goal))
    telos_agent.step_simulation()
    sleep(1.0/60.0)